### Run the following chunk of code to import any necessary libraries or packages needed for the rest of the script.

In [ ]:
import os as os
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv2
import tensorflow as tf
from scipy.ndimage import label
from skimage import measure

### Run the following chunk of code to extract possible "cells" for manual classification.

The user will have to change the channel_Directory, network_File_Path, and region_Save_Location variables to link to their relevant directories. Notice here how we use the phase one network to identify potential cell candidates, which we will then classify by hand later.

In [ ]:
channel_Directory = ""
network_File_Path = ""
region_Save_Location = ""

network = tf.keras.models.load_model(network_File_Path)

def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

def recolor_Image(img):
    maximum = np.mean(img) + 4*np.std(img)
    minimum = np.mean(img) - 4*np.std(img)
    color_Adjusted_Image = (img - minimum)/(maximum - minimum)
    color_Adjusted_Image[color_Adjusted_Image < 0] = 0
    color_Adjusted_Image[color_Adjusted_Image > 1] = 1
    return color_Adjusted_Image

count = 0
region_Number = 0
cell_Capture_Range = 40
for image_Name in os.listdir(channel_Directory):
    print("Analyzing " + image_Name[:-4])
    full_Channel = plt.imread(channel_Directory + image_Name)
    if np.max(full_Channel) == int(np.max(full_Channel)) and len(str(np.max(full_Channel))) == len(str(int(np.max(full_Channel)))):
        full_Channel = full_Channel.copy()/255.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel, cv2.COLOR_GRAY2RGB)
    if np.shape(full_Channel)[2] == 4:
        full_Channel = full_Channel.copy()[:,:,0:3]
    full_Channel = recolor_Image(full_Channel.copy())
    image_Height, image_Width, channels = np.shape(full_Channel)
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    full_Channel_Resized[full_Channel_Resized < 0] = 0
    full_Channel_Resized[full_Channel_Resized > 1] = 1
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))

    x_Slider = 0
    y_Slider = 0
    output_Array = np.zeros((128,128))
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            output = network.predict(current_Tile_Normalized)

            for i in range(128):
                for j in range(128):
                    output_Array[i,j] = np.argmax(output[0,i,j,:])
            
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    for i in range(image_Height_Resized):
        for j in range(image_Width_Resized):
            if output_Image[i,j] != 0:
                output_Image[i,j] = 1
            else:
                continue

    blobs, number_Of_Blobs = label(output_Image)
    properties = measure.regionprops(blobs)
    
    for prop in properties:
        count = count + 1
        if round(prop.centroid[0]) < cell_Capture_Range:
            if round(prop.centroid[1]) < cell_Capture_Range:
                region = full_Channel_Resized[0:cell_Capture_Range,0:cell_Capture_Range]  
            elif image_Width_Resized - prop.centroid[1] < cell_Capture_Range:
                region = full_Channel_Resized[0:cell_Capture_Range,image_Width_Resized - cell_Capture_Range:image_Width_Resized]   
            else:
                region = full_Channel_Resized[0:cell_Capture_Range,int(round(prop.centroid[1]) - cell_Capture_Range/2):int(round(prop.centroid[1]) + cell_Capture_Range/2)]
        elif round(prop.centroid[1]) < cell_Capture_Range:
            if image_Height_Resized - prop.centroid[0] < cell_Capture_Range:
                region = full_Channel_Resized[image_Height_Resized - cell_Capture_Range:image_Height_Resized,0:cell_Capture_Range]
            else:
                region = full_Channel_Resized[int(round(prop.centroid[0]) - cell_Capture_Range/2):int(round(prop.centroid[0]) + cell_Capture_Range/2),0:cell_Capture_Range]
        elif image_Height_Resized - prop.centroid[0] < cell_Capture_Range:
            if image_Width_Resized - prop.centroid[1] < cell_Capture_Range:
                region = full_Channel_Resized[image_Height_Resized - cell_Capture_Range:image_Height_Resized,image_Width_Resized - cell_Capture_Range:image_Width_Resized]
            else:
                region = full_Channel_Resized[image_Height_Resized - cell_Capture_Range:image_Height_Resized,int(round(prop.centroid[1]) - cell_Capture_Range/2):int(round(prop.centroid[1]) + cell_Capture_Range/2)]              
        elif image_Width_Resized - prop.centroid[1] < cell_Capture_Range:
                region = full_Channel_Resized[int(round(prop.centroid[0]) - cell_Capture_Range/2):int(round(prop.centroid[0]) + cell_Capture_Range/2),image_Width_Resized - cell_Capture_Range:image_Width_Resized]            
        else:
                region = full_Channel_Resized[int(round(prop.centroid[0]) - cell_Capture_Range/2):int(round(prop.centroid[0]) + cell_Capture_Range/2),int(round(prop.centroid[1]) - cell_Capture_Range/2):int(round(prop.centroid[1]) + cell_Capture_Range/2)]               
        plt.imsave(region_Save_Location + "/Region_" + str(count) + ".png",region)